### Our goal here is to combine our three datasets of interest and do t-tests/regressions by borough to see if the citywide trend holds true. So, we need to do the following:
#### 1) Build our yearly FHV trips by borough (2015-2017), using FHV Trip Data and taxi_zone_lookup
#### 2) Import and shape our yearly bus count by borough (2015-2017)
#### 3) Build our median income by borough from ACS
   

In [1]:
import pandas as pd
url_lookup = "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"

In [2]:
dfz = pd.read_csv(url_lookup)
dfz[:2]

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone


In [3]:
#Remove first row and last rows as non-NYC or unknown zones, as we did before
dfz = dfz[1:263]
dfz = dfz.loc [:,['LocationID', 'Borough']]
dfz[:2]

,LocationID,Borough
1,2,Queens
2,3,Bronx


In [4]:
#Concatenate url with month number and .csv, i.e. "1.csv"
#2015, 2016 FHV data has one locationid parameter
url_15 = "https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-" 
url_16 = "https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2016-"
url_17 = "https://data.cityofnewyork.us/resource/gx8w-pdsh.csv?$select=count(*)&$where=PULocationID=";
def get_count(zone_id):
    return pd.read_csv(url_17 + str(zone_id))['count'][0]

In [5]:
dfz['2017 Data'] = dfz['LocationID'].apply(get_count)
dfz[:3]

,LocationID,Borough,2017 Data
1,2,Queens,186
2,3,Bronx,94906
3,4,Manhattan,397327


In [6]:
#Experiencing throttling, size issues with 2015 and 2016 trips data
#2016 data was sorted and rolled up using the Open Data Socrata API
df16 = pd.read_csv('2016_For_Hire_Vehicle_Trip_Data.csv')

In [7]:
len(df16)

264

In [8]:
df16[:5]

,locationID,Dispatching_base_num
0,1.0,"9,334"
1,10.0,"111,822"
2,100.0,"853,261"
3,101.0,"24,013"
4,102.0,"222,993"


In [9]:
df16 = df16[1:]
df16 = df16.sort_values(by = ['locationID'])
df16.reset_index(inplace = True)
df16[:5]

,index,locationID,Dispatching_base_num
0,109,2.0,823
1,186,3.0,"97,826"
2,197,4.0,"503,140"
3,208,5.0,"5,560"
4,219,6.0,"15,651"


In [10]:
df16[-5:]

,index,locationID,Dispatching_base_num
258,179,262.0,"727,498"
259,180,263.0,"870,039"
260,181,264.0,"378,825"
261,182,265.0,"319,221"
262,263,NaN,"40,673,589"


In [11]:
#TLC reports zones 264, 265 as UNKNOWN location, so we drop these rows
df16 = df16[:260]

In [12]:
df16.set_index('locationID', inplace = True)

In [13]:
dfz.set_index('LocationID', inplace = True)

In [14]:
dfz['2016 Data'] = df16['Dispatching_base_num']
dfz[:5]

,Borough,2017 Data,2016 Data
LocationID,,,
2,Queens,186,823
3,Bronx,94906,"97,826"
4,Manhattan,397327,"503,140"
5,Staten Island,9848,"5,560"
6,Staten Island,20549,"15,651"


In [15]:
#Checking for null rows
dfz[dfz['2016 Data'].isnull()]

,Borough,2017 Data,2016 Data
LocationID,,,
103,Manhattan,0,NaN
104,Manhattan,0,NaN


In [16]:
#TLC zones 103-105 are Governor's/Ellis/Liberty Island, so we drop them. No harm, no foul.
dfz.loc[105]

Borough      Manhattan
2017 Data           23
2016 Data           54
Name: 105, dtype: object

In [17]:
dfz = dfz.drop([103,104,105])

In [18]:
dfz.loc[100:110]

,Borough,2017 Data,2016 Data
LocationID,,,
100,Manhattan,653769,"853,261"
101,Queens,32912,"24,013"
102,Queens,149348,"222,993"
106,Brooklyn,193577,"253,809"
107,Manhattan,948457,"1,259,880"
108,Brooklyn,83198,"61,940"
109,Staten Island,25343,"12,934"
110,Staten Island,126,59


In [19]:
#Our 2015 data can be found from the 2016 TLC Factbook dataset containing daily trips by borough
df15 = pd.read_excel('2016_tlc_factbook.xlsx', sheet_name='5_Trips_By_Borough')
df15

,Daily Average Trips by Borough,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Medallion Taxis & SHLs: Jan to Dec 2015,NaN,NaN,NaN,NaN
1,App-Based FHVs: Jul to Dec 2015,NaN,NaN,NaN,NaN
2,Traditional FHVs: Oct to Dec 2015,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,Medallion Taxi,SHL,App-Based FHV,Traditional FHV
5,Manhattan,362870,14937.1,101626,9962.25
6,Bronx,317.995,3380.46,3082.3,21496.6
7,Brooklyn,7537.91,19752.2,26249.9,8908.38
8,Queens*,4668.33,14487,9982.38,8730.25
9,Staten Island,5.72055,6.98904,241.668,36.3152


In [20]:
df15 = df15[4:10]
df15

,Daily Average Trips by Borough,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
4,NaN,Medallion Taxi,SHL,App-Based FHV,Traditional FHV
5,Manhattan,362870,14937.1,101626,9962.25
6,Bronx,317.995,3380.46,3082.3,21496.6
7,Brooklyn,7537.91,19752.2,26249.9,8908.38
8,Queens*,4668.33,14487,9982.38,8730.25
9,Staten Island,5.72055,6.98904,241.668,36.3152


In [21]:
df15 = df15.loc[:,['Daily Average Trips by Borough',"Unnamed: 3","Unnamed: 4"]]

In [22]:
df15['2015 Daily'] = df15[df15.columns[1]] + df15[df15.columns[2]]
df15

,Daily Average Trips by Borough,Unnamed: 3,Unnamed: 4,2015 Daily
4,NaN,App-Based FHV,Traditional FHV,App-Based FHVTraditional FHV
5,Manhattan,101626,9962.25,111588
6,Bronx,3082.3,21496.6,24578.9
7,Brooklyn,26249.9,8908.38,35158.3
8,Queens*,9982.38,8730.25,18712.6
9,Staten Island,241.668,36.3152,277.984


In [23]:
df15 = df15[1:]
df15 = df15[[df15.columns[0],df15.columns[3]]]
df15

,Daily Average Trips by Borough,2015 Daily
5,Manhattan,111588
6,Bronx,24578.9
7,Brooklyn,35158.3
8,Queens*,18712.6
9,Staten Island,277.984


In [24]:
df15.loc[8][0] = 'Queens'
df15

,Daily Average Trips by Borough,2015 Daily
5,Manhattan,111588
6,Bronx,24578.9
7,Brooklyn,35158.3
8,Queens,18712.6
9,Staten Island,277.984


In [54]:
#2016 data needs to be casted to int
dfz['2016 Data'] = dfz['2016 Data'].apply(lambda x: int(x.replace(',','')))

In [56]:
fhv = dfz.groupby('Borough').sum()

In [59]:
df15.set_index(df15.columns[0], inplace = True)
df15

,2015 Daily
Daily Average Trips by Borough,
Manhattan,111588
Bronx,24578.9
Brooklyn,35158.3
Queens,18712.6
Staten Island,277.984


In [61]:
#Multiply by 365 to get the year's trips count
df15 = df15.apply(lambda x: 365 * x)

In [62]:
fhv['2015 Data'] = df15
fhv[:5]

,2017 Data,2016 Data,2015 Data
Borough,,,
Bronx,4972794,4096583,8.97131e+06
Brooklyn,19601947,20551956,1.28328e+07
Manhattan,41150664,53682799,4.07295e+07
Queens,11899252,12687578,6.83011e+06
Staten Island,550990,430712,101464


In [81]:
#This dataset has several gaps: 2017 data has 75 mil despite 200 mil rows total, and 2016 data has 90 mil despite 135 mil rows
fhv.to_csv('yearly_tripbyboro.csv')

In [84]:
dfb = pd.read_csv('yearly_busbyboro.csv')
bus = dfb.loc[:,['Route','2015','2016','2017']]
bus

,Route,2015,2016,2017
0,Brooklyn Total,615477,612752,581227
1,Bronx Total,538711,522826,483663
2,Manhattan Total,424707,410479,380075
3,Queens Total,358402,359768,350548
4,Staten Is Total,128997,128042,123960
5,Grand Total,2070386,2038119,1923993


In [85]:
bus.rename(columns={'Route':'Borough'}, inplace = True)

In [86]:
bus['Borough'] = bus['Borough'].apply(lambda x: x[0:len(x)-5])

In [87]:
temp = bus.iloc[0].copy()
bus.iloc[0] = bus.iloc[1]
bus.iloc[1] = temp
bus[:5]

,Borough,2015,2016,2017
0,Bronx,538711,522826,483663
1,Brooklyn,615477,612752,581227
2,Manhattan,424707,410479,380075
3,Queens,358402,359768,350548
4,Staten Is,128997,128042,123960


In [101]:
fhv.index[0]

'Bronx'

In [118]:
dataset = pd.DataFrame(columns = ['fhv','boro','bus'])
for i in range(0,5):
    dataset.loc[i] = [fhv['2015 Data'][i], fhv.index[i], bus['2015'][i]]
dataset

,fhv,boro,bus
0,8.971309e+06,Bronx,538711
1,1.283277e+07,Brooklyn,615477
2,4.072954e+07,Manhattan,424707
3,6.830110e+06,Queens,358402
4,1.014640e+05,Staten Island,128997


In [119]:
for i in range(5,10):
    dataset.loc[i] = [fhv['2016 Data'][i-5], fhv.index[i-5], bus['2016'][i-5]]
for i in range(10,15):
    dataset.loc[i] = [fhv['2017 Data'][i-10], fhv.index[i-10], bus['2017'][i-10]]
dataset

,fhv,boro,bus
0,8.971309e+06,Bronx,538711
1,1.283277e+07,Brooklyn,615477
2,4.072954e+07,Manhattan,424707
3,6.830110e+06,Queens,358402
4,1.014640e+05,Staten Island,128997
5,4.096583e+06,Bronx,522826
6,2.055196e+07,Brooklyn,612752
7,5.368280e+07,Manhattan,410479
8,1.268758e+07,Queens,359768
9,4.307120e+05,Staten Island,128042


In [120]:
X = dataset[['fhv','boro']].values
X

array([[8971309.211956521, 'Bronx'],
       [12832773.152173912, 'Brooklyn'],
       [40729540.652173914, 'Manhattan'],
       [6830110.108695652, 'Queens'],
       [101464.04891304349, 'Staten Island'],
       [4096583.0, 'Bronx'],
       [20551956.0, 'Brooklyn'],
       [53682799.0, 'Manhattan'],
       [12687578.0, 'Queens'],
       [430712.0, 'Staten Island'],
       [4972794.0, 'Bronx'],
       [19601947.0, 'Brooklyn'],
       [41150664.0, 'Manhattan'],
       [11899252.0, 'Queens'],
       [550990.0, 'Staten Island']], dtype=object)

In [121]:
y = dataset['bus'].values
y

array([538711, 615477, 424707, 358402, 128997, 522826, 612752, 410479,
       359768, 128042, 483663, 581227, 380075, 350548, 123960],
      dtype=object)

In [123]:
dataset.to_csv('training_dataset.csv')